In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Neural Networks

/content/drive/.shortcut-targets-by-id/1ykFg7g41RO1OmduNF4cgF7d9se6qdaa4/Neural Networks


In [ ]:
#Retrieving Dataset
train_data = pd.read_csv('UNSW_NB15_training-set.csv')
test_data = pd.read_csv('UNSW_NB15_testing-set.csv')
train_data

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,1.415894e+04,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8.395112e+03,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1.572272e+03,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8.561499e+03,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,175337,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,0,0.009000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,57,0,0,0,24,2,24,24,13,24,0,0,0,24,24,0,Generic,1
175337,175338,0.505762,tcp,-,FIN,10,8,620,354,33.612649,254,252,8.826286e+03,4903.492188,2,1,54.400111,66.980570,3721.068786,120.177727,255,3518776216,3453092386,255,0.099440,0.036895,0.062545,62,44,0,0,1,1,1,1,1,2,0,0,0,1,1,0,Shellcode,1
175338,175339,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,0,0.009000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,57,0,0,0,12,2,3,3,3,13,0,0,0,3,12,0,Generic,1
175339,175340,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,0,0.009000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,57,0,0,0,30,2,30,30,14,30,0,0,0,30,30,0,Generic,1


In [ ]:
#Using Label Encoders for columns labels 'proto','service','state','attack_cat' as they contain string values
#After applying Label Encoder
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
train_data['proto']= label_encoder.fit_transform(train_data['proto'])  
train_data['service']= label_encoder.fit_transform(train_data['service']) 
train_data['state']= label_encoder.fit_transform(train_data['state']) 
train_data['attack_cat']= label_encoder.fit_transform(train_data['attack_cat'])


test_data['proto']= label_encoder.fit_transform(test_data['proto'])  
test_data['service']= label_encoder.fit_transform(test_data['service']) 
test_data['state']= label_encoder.fit_transform(test_data['state']) 
test_data['attack_cat']= label_encoder.fit_transform(test_data['attack_cat'])

In [ ]:
#droping column 'id'
train_data=train_data.drop(columns=['id'],axis=1)
test_data=test_data.drop(columns=['id'],axis=1)

In [ ]:
#Normalizing Data
labels = ['dur','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_srv_dst','is_sm_ips_ports','label']
for each in labels:
  train_data[each] = train_data[each]/train_data[each].max()
  test_data[each] = test_data[each]/test_data[each].max()

In [ ]:
X_train=np.array(train_data)[:,1:-1]
X_test=np.array(test_data)[:,1:-1]
y_train=np.array(train_data)[:,-1]
y_test=np.array(test_data)[:,-1]

In [ ]:
print(y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(X_test.shape)

(175341,)
(82332,)
(175341, 42)
(82332, 42)


In [ ]:
X_train_0 = X_train[:1000,:]
X_train_1 = X_train[-1000:,:]

X_test_0 = X_test[:100,:]
X_test_1 = X_test[-100:,:]


y_train_0 = y_train[:1000]
y_train_1 = y_train[-1000:]

y_test_0 = y_test[:100]
y_test_1 = y_test[-100:]


X_train = np.concatenate((X_train_0,X_train_1),axis = 0)
y_train = np.concatenate((y_train_0,y_train_1),axis = 0)
X_test = np.concatenate((X_test_0,X_test_1),axis = 0)
y_test = np.concatenate((y_test_0,y_test_1),axis = 0)


print(y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(X_test.shape)

(2000,)
(200,)
(2000, 42)
(200, 42)


In [ ]:
# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(42,), n_classes=2):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes,10)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] 
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0],))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# output
	out_layer = Dense(1, activation='sigmoid')(merge)
	print(merge.shape, li.shape, in_image.shape)
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
d_model=define_discriminator()
print()

(None, 84) (None, 42) (None, 42)



In [ ]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=2):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 10)(in_label)
	# linear multiplication
	n_nodes = 42
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((42,))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	# n_nodes = 128 * 7 * 7
	# gen = Dense(n_nodes)(in_lat)
	# gen = LeakyReLU(alpha=0.2)(gen)
	# gen = Reshape((7, 7, 128))(gen)
	# merge image gen and label input
	merge = Concatenate()([in_lat, li])
	#print(merge.shape, li.shape, in_lat.shape)
	# gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
	# gen = LeakyReLU(alpha=0.2)(gen)
	# # upsample to 28x28
	# gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	# gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Dense(42, activation='tanh')(merge)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

In [ ]:
g_model=define_generator(42)
gen_noise, gen_label = g_model.input
gen_output = g_model.output
#gan_output = d_model([gen_output, gen_label])
print("gen_noise:",gen_noise.shape,"gen_label:", gen_label.shape)
print("gen_output:",gen_output.shape)
#print("gan_output:",gan_output.shape)

gen_noise: (None, 42) gen_label: (None, 1)
gen_output: (None, 42)


In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	print(gen_noise.shape, gen_label.shape)
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load fashion mnist images
def load_real_samples():
	# load dataset
	trainX, trainy = X_train, y_train
	# # expand to 3d, e.g. add channels
	# X = expand_dims(trainX, axis=-1)
	# # convert from ints to floats
	# X = X.astype('float32')
	# # scale from [0,255] to [-1,1]
	# X = (X - 127.5) / 127.5
	return [trainX, trainy]

# # select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=2):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(1):
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
			# generate 'fake' examples
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
			# prepare points in latent space as input for the generator
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
	# save the generator model
	g_model.save('cgan_generator_unsw.h5')

# size of the latent space
latent_dim = 42
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

(None, 84) (None, 42) (None, 42)
(None, 42) (None, 1)
>1, 1/15, d1=0.006, d2=0.690 g=0.736
>2, 1/15, d1=0.000, d2=0.726 g=0.724
>3, 1/15, d1=0.000, d2=0.706 g=0.726
>4, 1/15, d1=0.005, d2=0.742 g=0.701
>5, 1/15, d1=0.006, d2=0.724 g=0.730
>6, 1/15, d1=0.000, d2=0.713 g=0.753
>7, 1/15, d1=0.000, d2=0.731 g=0.736
>8, 1/15, d1=0.006, d2=0.754 g=0.703
>9, 1/15, d1=0.006, d2=0.683 g=0.720
>10, 1/15, d1=0.000, d2=0.635 g=0.743
>11, 1/15, d1=0.000, d2=0.703 g=0.718
>12, 1/15, d1=0.000, d2=0.778 g=0.766
>13, 1/15, d1=0.000, d2=0.731 g=0.743
>14, 1/15, d1=0.000, d2=0.675 g=0.716
>15, 1/15, d1=0.000, d2=0.674 g=0.726
>16, 1/15, d1=0.000, d2=0.703 g=0.707
>17, 1/15, d1=0.000, d2=0.645 g=0.725
>18, 1/15, d1=0.000, d2=0.687 g=0.760
>19, 1/15, d1=0.000, d2=0.755 g=0.710
>20, 1/15, d1=0.000, d2=0.738 g=0.746
>21, 1/15, d1=0.000, d2=0.721 g=0.734
>22, 1/15, d1=0.000, d2=0.667 g=0.717
>23, 1/15, d1=0.000, d2=0.688 g=0.746
>24, 1/15, d1=0.000, d2=0.699 g=0.748
>25, 1/15, d1=0.006, d2=0.695 g=0.751
>26, 

In [ ]:
# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=2):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# load model
model = load_model('cgan_generator_unsw.h5')
# generate images
latent_points, labels = generate_latent_points(42, 5000)
labels = asarray([x for _ in range(2500) for x in range(2)])
# generate images
print(labels)
X  = model.predict([latent_points, labels])
print(X)

[0 1 0 ... 1 0 1]
[[-0.6265656  -0.20802519  0.26915672 ... -0.94047    -0.6238031
   0.37395307]
 [ 0.62700003  0.04426846 -0.05665417 ... -0.26404986 -0.03122857
  -0.72369576]
 [-0.5265586   0.8260895   0.39273998 ...  0.5766172  -0.23850647
  -0.32512063]
 ...
 [-0.72016704  0.93846184 -0.24920675 ... -0.76399344  0.8165879
  -0.86530316]
 [-0.18276016 -0.21761012  0.872786   ...  0.89451087 -0.62519664
   0.3040294 ]
 [-0.35437536 -0.18480217 -0.22210559 ... -0.46577525 -0.49357
   0.56336695]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(max_depth=2, random_state=0)
Y=[0 for i in range(2500)]
for i in range(2500):
  Y.append(1)
clf.fit(X,Y)
print("Test Data Overall Accuracy: ",accuracy_score(y_test,clf.predict(X_test)))

Test Data Overall Accuracy:  0.81
